# End-to-end

This notebook should form the core skeleton of the 'run' function

## Colab Set up

In [1]:
# -- env setup
import os
import gc

!python3.7 -m pip install git+https://github.com/namiyousef/colab-utils.git
from colabtools.utils import get_gpu_utilization, mount_drive, install_private_library

drive_path = mount_drive()
project_path = os.path.join(drive_path, 'argument-mining')
development_dir = os.path.join(drive_path, 'argument-mining/argminer')

install_private_library(os.path.join(project_path, 'data/github_config.json'), 'argument-mining')

  Cloning https://github.com/namiyousef/colab-utils.git to /tmp/pip-req-build-5i7ydzp6
  Running command git clone -q https://github.com/namiyousef/colab-utils.git /tmp/pip-req-build-5i7ydzp6
  Created wheel for colabtools: filename=colabtools-0.0.5-py3-none-any.whl size=3585 sha256=b0808ec10b9aba31db65c105148d19c652618702710623944c7f38fcac29e020
  Stored in directory: /tmp/pip-ephem-wheel-cache-t4irkesa/wheels/1c/35/c0/364531e4ff0f0fe0f3296c80f1ee668b03ae6c6c378c5a44bf
Successfully built colabtools
Google Drive import successful.
CUDA device detected. Using GPU...
Mounted at /content/drive
Google Drive mount successful.


### Imports

In [2]:
# -- public imports

from transformers import AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
from torch.utils.data import DataLoader
import torch
from pandas.testing import assert_frame_equal
import time

# -- private imports
from colabtools.utils import move_to_device
from colabtools.config import DEVICE

# -- dev imports
%load_ext autoreload
%autoreload 2

from argminer.data import ArgumentMiningDataset, TUDarmstadtProcessor, PersuadeProcessor
from argminer.evaluation import inference
from argminer.utils import encode_model_name
from argminer.config import LABELS_MAP_DICT



Import of ArgMiner successful


In [11]:
# constants (these will be abstracted away by inputs that you give to run)

# -- model specific configurations
model_name = 'google/bigbird-roberta-base'
max_length = 1024

# -- training configurations
epochs = 5
batch_size = 2
verbose = 2
save_freq = 2

# -- dataset configurations
dataset_name = 'Persuade'

# -- experiment configurations
strategy = 'standard_bieo'
strat_name, strat_label = strategy.split('_')

# -- inferred configurations
df_label_map = LABELS_MAP_DICT[dataset_name][strat_label]
num_labels = len(set(df_label_map.label))
Processor = eval(f'{dataset_name}Processor')


### Tokenizer, Model and Optimizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels) 
# TODO force_download
# TODO add option for optimizer
optimizer = torch.optim.Adam(params=model.parameters())

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were no

### Dataset 
Note this will change as the Processor develops. On the cluster you will need to use different options

In [16]:
processor = Processor(os.path.join(project_path, f'data/{strat_label}'))
processor = processor.from_json()
df_total = processor.dataframe

df_dict = processor.get_tts(test_size=0.3, val_size=0.1)
df_train = df_dict.get('train')[['text', 'labels']]
df_test = df_dict.get('test')[['text', 'labels']]
df_val = df_dict.get('val')[['text', 'labels']]

#df_train = df_total[['text', 'labels']].head(10) 
#df_test = df_total[['text', 'labels']].tail(201)


#assert_frame_equal(df_total[['text', 'labels']], pd.concat([df_train, df_test]))

# todo this changes NOTE FIXED BT STRATEGY!!
# todo this needs to get updated as well.....


<class 'argminer.data.PersuadeProcessor'>


In [14]:
train_set = ArgumentMiningDataset(df_label_map, df_train, tokenizer, max_length, strategy)
test_set = ArgumentMiningDataset(df_label_map, df_test, tokenizer, max_length, strategy, is_train=False)

train_loader = DataLoader(train_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

In [15]:
if not os.path.exists('models'):
  os.makedirs('models')
  print('models directory created!')
model.to(DEVICE)
print(f'Model pushed to device: {DEVICE}')
for epoch in range(epochs):
    model.train()
    start_epoch_message = f'EPOCH {epoch + 1} STARTED'
    print(start_epoch_message)
    print(f'{"-" * len(start_epoch_message)}')
    start_epoch = time.time()

    start_load = time.time()
    training_loss = 0
    for i, (inputs, targets) in enumerate(train_loader):
        start_train = time.time()
        inputs = move_to_device(inputs, DEVICE)
        targets = move_to_device(targets, DEVICE)
        if DEVICE != 'cpu':
            print(f'GPU Utilisation at batch {i+1} after data loading: {get_gpu_utilization()}')

        optimizer.zero_grad()

        loss, outputs = model(
            labels=targets,
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            return_dict=False
        )
        if DEVICE != 'cpu':
            print(f'GPU Utilisation at batch {i+1} after training: {get_gpu_utilization()}')


        training_loss += loss.item()

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        del targets, inputs, loss, outputs
        gc.collect()
        torch.cuda.empty_cache()

        end_train = time.time()

        if verbose > 1:
            print(
                f'Batch {i + 1} complete. Time taken: load({start_train - start_load:.3g}), '
                f'train({end_train - start_train:.3g}), total({end_train - start_load:.3g}). '
            )
        start_load = time.time()

    print_message = f'Epoch {epoch + 1}/{epochs} complete. ' \
                    f'Time taken: {start_load - start_epoch:.3g}. ' \
                    f'Loss: {training_loss/(i+1): .3g}'

    if verbose:
        print(f'{"-" * len(print_message)}')
        print(print_message)
        print(f'{"-" * len(print_message)}')

    if epoch % save_freq == 0:
        encoded_model_name = encode_model_name(model_name, epoch+1)
        save_path = f'models/{encoded_model_name}'
        model.save_pretrained(save_path)
        print(f'Model saved at epoch {epoch+1} at: {save_path}')

encoded_model_name = encode_model_name(model_name, 'final')
save_path = f'models/{encoded_model_name}'
model.save_pretrained(save_path)
print(f'Model saved at epoch {epoch + 1} at: {save_path}')

Model pushed to device: cuda
EPOCH 1 STARTED
---------------
GPU Utilisation at batch 1 after data loading: 2739


/usr/local/lib/python3.7/dist-packages/transformers/models/big_bird/modeling_big_bird.py:978: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  * num_indices_to_pick_from


GPU Utilisation at batch 1 after training: 5763
Batch 1 complete. Time taken: load(0.0248), train(2.19), total(2.22). 
GPU Utilisation at batch 2 after data loading: 2797
GPU Utilisation at batch 2 after training: 7251
Batch 2 complete. Time taken: load(0.0156), train(2.04), total(2.05). 
GPU Utilisation at batch 3 after data loading: 2797
GPU Utilisation at batch 3 after training: 7251
Batch 3 complete. Time taken: load(0.0133), train(2.03), total(2.04). 
GPU Utilisation at batch 4 after data loading: 2797
GPU Utilisation at batch 4 after training: 7251
Batch 4 complete. Time taken: load(0.0162), train(2.04), total(2.06). 
GPU Utilisation at batch 5 after data loading: 2797
GPU Utilisation at batch 5 after training: 7251
Batch 5 complete. Time taken: load(0.0172), train(2.03), total(2.04). 
GPU Utilisation at batch 6 after data loading: 2797
GPU Utilisation at batch 6 after training: 7251
Batch 6 complete. Time taken: load(0.024), train(2.02), total(2.05). 
GPU Utilisation at batch 7 

KeyboardInterrupt: ignored

In [ ]:
# load trained model
path = ''
trained_model = AutoModelForTokenClassification.from_pretrained(path)

In [ ]:
df_metrics, df_scores = inference(trained_model, test_loader)

In [ ]:
df_scores